# Different Tool Configuration Methods

This notebook demonstrates different approaches to configuring tools for LangChain agents, from simple lists to advanced error handling.

## Key Concepts
- **Method 1**: Simple list of tools (quick setup)
- **Method 2**: ToolNode with custom configuration (production-ready)
- **Error Handling**: Making agents robust to tool failures

## Why Tool Configuration Matters
- **Without error handling**: Errors crash the agent
- **With error handling**: Errors become feedback for the model
- **Model adaptation**: Agent can retry or adjust approach based on error messages

In [1]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, ToolNode
import tools

## Method 1: Simple List of Tools

The simplest approach - just pass tools as a list. LangChain handles everything automatically.

In [2]:
print("=== Method 1: Simple List of Tools ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: Pass list of tools (simple approach)
simple_tools = [tools.web_search, tools.calculate]
agent1 = create_agent(
    model, 
    tools=simple_tools
)

print("✓ Agent 1 created with simple tool list")
print("  - LangChain automatically creates ToolNode")
print("  - Default error handling behavior")
print("  - Good for prototyping and simple use cases")

=== Method 1: Simple List of Tools ===
✓ Agent 1 created with simple tool list
  - LangChain automatically creates ToolNode
  - Default error handling behavior
  - Good for prototyping and simple use cases


In [3]:
simple_tools[1].invoke({"expression": "What is 123 * 456?"})

'Error: Invalid mathematical expression'

## Method 2: Advanced ToolNode Configuration

Create a ToolNode explicitly with custom settings for better control over tool execution.

In [4]:
print("=== Method 2: Advanced ToolNode Configuration ===")

# Method 2: Use ToolNode with error handling (advanced approach)
tool_node = ToolNode(
    tools=[tools.web_search, tools.calculate],
    handle_tool_errors="Please check your input and try again. Error details will help you correct the issue."
)

agent2 = create_agent(model, tools=tool_node)

print("✓ Agent 2 created with advanced ToolNode")
print("  - Custom error handling messages")
print("  - Better control over tool execution")
print("  - Production-ready error recovery")

=== Method 2: Advanced ToolNode Configuration ===
✓ Agent 2 created with advanced ToolNode
  - Custom error handling messages
  - Better control over tool execution
  - Production-ready error recovery


In [5]:
# Unlike tools.calculate.invoke() which directly calls the tool, 
# ToolNode.invoke() requires a state dictionary with messages containing tool_calls. 
# This is because ToolNode is designed to work within a LangGraph agent workflow where messages follow this structure.
from langchain_core.messages import AIMessage

state_with_invalid_args = {
    "messages": [
        AIMessage(
            content="",
            tool_calls=[
                {
                    "name": "calculate",
                    "args": {"input": "What is 123 * 456?"},
                    "id": "call_456",
                    "type": "tool_call"
                }
            ]
        )
    ]
}

result = tool_node.invoke(state_with_invalid_args)
print(result)

{'messages': [ToolMessage(content='Please check your input and try again. Error details will help you correct the issue.', name='calculate', tool_call_id='call_456', status='error')]}


## Comparing Both Approaches

Let's test both agents with the same query to see how they behave:

In [6]:
# Test query that uses multiple tools
test_query = "Search for Python tutorials and calculate 10 times 5"

print(f"Testing both agents with: '{test_query}'")
print("=" * 60)

# Test Agent 1 (Simple)
print("\n=== Agent 1 Results (Simple Tool List) ===")
try:
    result1 = agent1.invoke({"messages": test_query})
    print(f"✓ Response: {result1['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

# Test Agent 2 (Advanced)
print("\n=== Agent 2 Results (Advanced ToolNode) ===")
try:
    result2 = agent2.invoke({"messages": test_query})
    print(f"✓ Response: {result2['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

print("\n💡 Both agents work the same for valid inputs, but Agent 2 handles errors better")

Testing both agents with: 'Search for Python tutorials and calculate 10 times 5'

=== Agent 1 Results (Simple Tool List) ===
✓ Response: <think>
Okay, let me see. The user asked to search for Python tutorials and calculate 10 times 5. I called both functions as instructed. The web search returned five results, which I formatted with titles, descriptions, and URLs. Then, the calculation was straightforward, 10 multiplied by 5 equals 50. I need to present both results clearly. The user might be a beginner looking for tutorials and also needs a quick math answer. I should make sure the search results are easy to read and the calculation is clearly stated. No errors in the responses, so just format them properly and combine the answers.
</think>

Here are the results:

**Python Tutorials Search:**
1. [Free Python Libraries Course](https://www.simplilearn.com/learn-python-libraries-free-course-skillup) - Discover essential data science libraries
2. [YouTube Python Basics](https://www.reddit

In [7]:
def test_agent(agent, name, query):
    """Test an agent with a query and display results."""
    print(f"\n=== {name} ===")
    try:
        result = agent.invoke({"messages": query})
        print(f"✓ {result['messages'][-1].content}")
    except Exception as e:
        print(f"✗ Error: {e}")

# Test 1: Valid query
valid_query = "Search for Python tutorials and calculate 10 times 5"
print(f"Test 1 - Valid Query: '{valid_query}'")
print("=" * 60)
test_agent(agent1, "Simple Agent", valid_query)
test_agent(agent2, "Advanced Agent", valid_query)

# Test 2: Query that causes tool error
error_query = "Calculate 10 divided by 0"
print(f"\n\nTest 2 - Error Query: '{error_query}'")
print("=" * 60)
test_agent(agent1, "Simple Agent (crashes)", error_query)
test_agent(agent2, "Advanced Agent (handles gracefully)", error_query)



Test 1 - Valid Query: 'Search for Python tutorials and calculate 10 times 5'

=== Simple Agent ===
✓ <think>
Okay, let me process the user's request. They wanted two things: a search for Python tutorials and the calculation of 10 times 5.

First, I called the web_search function with "Python tutorials" to get relevant resources. The results included several free courses and tutorials, which I formatted in the response. Then, I used the calculate function with the expression "10 * 5" to perform the multiplication. The result was 50, which I presented clearly.

I made sure to separate the two parts of the answer, first listing the search results and then showing the calculation. The user might be a beginner looking to learn Python, so providing both the tutorials and the math answer covers their possible needs. I checked that the links from the search results are valid and relevant, and the calculation is straightforward. Everything seems to be in order.
</think>

Here are the results:

